In [1]:
### Need to add points allowed (to do this, probabily need to look into matchups)
from ipynb.fs.defs.ELO_Tracking import get_pre_elo
from ipynb.fs.defs.Modeling_Prep_2 import get_player_games_before, get_player_avgs_over_last_x_games, get_team_avgs_over_last_x_games, get_out_players_for_game, create_team_predictions, pregame_stat_2, create_predictdf_2

import nba_api
import pandas as pd
import sklearn
import itertools
import os
import numpy as np
import requests
import time
import random
from sklearn.model_selection import KFold
import datetime as dt
import seaborn as sns
from nba_api.stats.static import players
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta
from nba_api.stats.library.parameters import SeasonAllNullable
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.library.parameters import SeasonTypeNullable
from nba_api.stats.library.parameters import SeasonType
from nba_api.stats.library.parameters import SeasonTypePlayoffs
from nba_api.stats.library.parameters import  SeasonNullable
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
from tqdm import tqdm
from scipy.stats import norm
from nba_api.stats import endpoints
from nba_api.stats.static import teams
from sklearn.ensemble import RandomForestClassifier
import ipynb
import importlib
import warnings

## Also read in ELO functions and tracked data

ELO_track = pd.read_csv('/Users/nickbachelder/Buckets Model/ELO_track', index_col=0)
ELO_track['Date'] = pd.to_datetime(ELO_track['Date'])

from ipynb.fs.defs.ELO_Tracking import get_pre_elo


## Also read in cleaned NBA data
NBAgamesC = pd.read_csv('/Users/nickbachelder/Buckets Model/NBAgamesC', index_col=0)
NBAgamesC['GAME_DATE_H'] = pd.to_datetime(NBAgamesC['GAME_DATE_H'])
nba_team = pd.read_csv('/Users/nickbachelder/Buckets Model/nba_team', index_col=0)
NBAgames = pd.read_csv('/Users/nickbachelder/Buckets Model/NBAgames', index_col=0)
NBAgames['GAME_DATE'] = pd.to_datetime(NBAgames['GAME_DATE'])

## Also import roster data

from ipynb.fs.defs.Roster_Injury_Implementation import nested_dict_folder
unfolded_roster = pd.read_csv('/Users/nickbachelder/Buckets Model/unfolded_roster', index_col=0)
folded_roster = nested_dict_folder(unfolded_roster, nba_team)

## Also import out data
from ipynb.fs.defs.Roster_Injury_Implementation import get_curr_injuries
from ipynb.fs.defs.Roster_Injury_Implementation import nested_dict_folder
unfolded_out = pd.read_csv('/Users/nickbachelder/Buckets Model/unfolded_out', index_col=0)
folded_out = nested_dict_folder(unfolded_out, nba_team)

# Also read in all box data for each player since 2015
all_boxes = pd.read_csv('/Users/nickbachelder/Buckets Model/all_boxes', index_col=0)
all_boxes['GAME_DATE'] = pd.to_datetime(all_boxes['GAME_DATE'])
NBAgamesC['GAME_ID_H'] = NBAgamesC['GAME_ID_H'].astype(int)

100%|██████████| 22/22 [00:00<00:00, 127.33it/s]


In [2]:
## remove all new game / box stats from this season (they will be readded in script)

NBAgamesC = NBAgamesC[NBAgamesC['SEASON_ID_H'] != 2021]
all_boxes = all_boxes[all_boxes['SEASON_YEAR'] != '2021-22']
ELO_track = ELO_track[ELO_track['Season'] != 2021]

In [3]:
playoff = 0
curr_season = 2021
curr_season_char = '2021-22'

In [4]:
import warnings
warnings.simplefilter("ignore", UserWarning)

def get_next_20_games(playoff = 0):
    ## get next 20 games
    delta = dt.timedelta(days=1)
    upcoming_games = pd.DataFrame()
    num_games = 0
    today = int((str(datetime.today()).split()[0].replace('-','')))
    today_as_date = datetime.today()
    while num_games < 20:
        upcoming_games_html =  pd.read_html('https://www.espn.com/nba/schedule/_/date/' + str(today))[0]
        if 'matchup' in upcoming_games_html.columns:
            upcoming_games_html['GAME_DATE_H'] =  str(today_as_date) 
            upcoming_games = pd.concat([upcoming_games, upcoming_games_html])
            num_games += len(upcoming_games_html)
        today = today + 1
        today_as_date += delta
    upcoming_games = upcoming_games[~upcoming_games['matchup'].str.contains('East')] # remove non games
    upcoming_games = upcoming_games[~upcoming_games['matchup'].str.contains('West')].reset_index(drop = 1)
    upcoming_games['MATCHUP_H'] = ' ' + upcoming_games['Unnamed: 1'].str[-3:]  + ' v ' + upcoming_games['matchup'].str[-3:] + ' '
    upcoming_games['PLAYOFF_H'] = playoff
    
    ### Create prediction dataframe
    upcoming_games['MATCHUP_H'] = upcoming_games['MATCHUP_H'].str.replace(' SA ', ' SAS ')
    upcoming_games['MATCHUP_H'] = upcoming_games['MATCHUP_H'].str.replace('TAH', 'UTA')
    upcoming_games['MATCHUP_H'] = upcoming_games['MATCHUP_H'].str.replace('WSH', 'WAS')
    upcoming_games['MATCHUP_H'] = upcoming_games['MATCHUP_H'].str.replace(' NY ', ' NYK ')
    upcoming_games['MATCHUP_H'] = upcoming_games['MATCHUP_H'].str.replace(' NO ', ' NOP ')
    upcoming_games['MATCHUP_H'] = upcoming_games['MATCHUP_H'].str.replace(' GS ',' GSW ')
    upcoming_games['MATCHUP_H'] = upcoming_games['MATCHUP_H'].str.strip()
    upcoming_games = upcoming_games.rename(columns = {'matchup':'Away Team', 'Unnamed: 1':'Home Team'})
    upcoming_games['GAME_DATE_H'] = upcoming_games['GAME_DATE_H'].str[:10]
    return(upcoming_games)

upcoming_games = get_next_20_games()

In [5]:
warnings.simplefilter("ignore", RuntimeWarning)

predictdfTODAY = create_predictdf_2(sample = upcoming_games, NBAgames = NBAgames, 
                               NBAgamesC = NBAgamesC, nba_team = nba_team, ELO_track = ELO_track, 
                               all_boxes = all_boxes, num_games_for_avg = 10, folded_roster = folded_roster, folded_out = folded_out,
                                   upcoming = True, current_season = 2021)

predictdfTODAY

100%|██████████| 21/21 [00:06<00:00,  3.06it/s]


,FGA_H,FG3A_H,FTA_H,REB_H,AST_H,STL_H,BLK_H,FG_P_H,FG_P3_H,FT_P_H,ELO_H,PTS_H,AVG_PLAYOFF_MIN_H,P_WIN_H,AVG_MARGIN_H,AVG_PTS_ALLOWED_H,FGA_A,FG3A_A,FTA_A,REB_A,AST_A,STL_A,BLK_A,FG_P_A,FG_P3_A,FT_P_A,ELO_A,PTS_A,AVG_PLAYOFF_MIN_A,P_WIN_A,AVG_MARGIN_A,AVG_PTS_ALLOWED_A,ROAD_WIN_P_A,PLAYOFF,TEAM_PLAYOFF_GAMES_P5_H,TEAM_PLAYOFF_GAMES_P5_A
0,100.913,40.3673,28.6994,51.9938,34.9459,7.5243,3.83523,0.40474,0.307801,0.627422,1559,123.664,3223.71,0.6,-1.6,105.8,93.6854,36.5754,25.1337,47.2355,26.4301,8.67479,6.25872,0.457089,0.316589,0.582076,1633,121.041,2390.53,0.5,4.3,107.3,NaN,0,27,21
0,70.7928,27.013,13.8941,27.6843,13.7208,5.83228,3.41008,0.403754,0.291717,0.323209,1428,79.5875,293.242,0.1,-7.1,115.4,88.212,27.4023,29.1961,50.4074,22.2656,6.83754,5.15244,0.49622,0.384583,0.58855,1637,120.999,759.771,0.5,5.8,109.2,NaN,0,55,38
0,82.704,29.7232,25.861,39.4055,22.1578,6.03686,4.91464,0.396042,0.290136,0.56986,1487,101.63,1029.6,0.2,-6.6,118.2,73.3024,26.2807,14.5333,36.4173,14.9375,4.93849,5.95547,0.430189,0.303373,0.474191,1266,84.5761,204.007,0.2,-15.8,117.7,NaN,0,68,10
0,77.6433,33.3742,20.5868,42.1693,29.7794,6.99513,2.79773,0.420937,0.35464,0.62108,1512,99.5283,1567.19,0.5,-4.8,115.4,96.0453,37.8917,21.5268,43.173,26.1062,9.7107,3.33256,0.428883,0.301478,0.603296,1585,114.863,821.813,0.5,-3.4,108.6,NaN,0,30,24
0,86.5097,34.8795,26.084,48.8905,26.8251,9.10182,6.27872,0.469495,0.306102,0.571172,1408,112.35,261.63,0.5,4.1,117.6,81.743,29.8167,22.0872,50.1635,21.8524,5.19912,6.49667,0.479371,0.290103,0.57585,1469,106.364,228.404,0.4,-3,115.1,NaN,0,5,9
0,31.1772,5.92218,8.96737,28.3044,7.03082,4.24953,3.16284,0.438442,0.0743822,0.334933,1239,36.492,138.025,0.1,-19.3,121,89.5156,33.9968,16.578,50.298,21.2736,7.46337,5.28561,0.452245,0.340286,0.545769,1419,107.177,359.591,0.3,-4.9,112.1,NaN,0,23,0
0,86.7173,22.6123,15.2034,49.3929,22.2129,5.69237,4.60705,0.4497,0.281141,0.468425,1466,100.75,394.515,0.2,-9.2,119.5,96.4695,45.3855,26.2461,46.3374,22.4513,6.74957,4.62803,0.472033,0.432644,0.637045,1646,132.246,1206.3,0.6,-0.6,118.3,NaN,0,28,45
0,87.5996,35.5359,23.4774,46.9427,26.5907,7.11412,5.14075,0.480702,0.380782,0.687605,1601,119.065,1048.95,0.5,2.5,118.6,88.7948,42.18,21.6597,43.1613,32.197,7.40856,4.69086,0.453311,0.341621,0.691816,1543,118.903,1983.4,0.8,10.8,107.8,NaN,0,35,60
0,78.2664,29.387,18.316,50.5744,21.2598,9.75185,6.19399,0.426232,0.319659,0.387712,1432,94.8936,357.162,0.6,3.7,105.8,85.2734,26.172,17.2878,42.8195,21.16,6.94202,4.7914,0.444344,0.341994,0.561428,1694,102.515,1490.16,0.4,-1.1,107.5,NaN,0,0,22
0,89.5618,38.6686,24.185,39.9642,22.3021,7.22022,3.65596,0.45221,0.330466,0.617152,1659,115.239,1701.28,0.6,5,107.2,89.734,35.3061,20.4734,40.4847,22.1323,5.24986,3.19649,0.444038,0.377012,0.530877,1597,114.509,1005.73,0.4,-5.5,120.3,NaN,0,45,43


In [6]:
pd.DataFrame.to_csv(predictdfTODAY, '/Users/nickbachelder/Buckets Model/daily_data')

In [7]:
upcoming_games = upcoming_games[['Away Team', 'Home Team', 'GAME_DATE_H']]
upcoming_games

pd.DataFrame.to_csv(upcoming_games, '/Users/nickbachelder/Buckets Model/upcoming_games')

In [8]:
combos = list(itertools.combinations(nba_team['abbreviation'], 2))
combos_backward = [t[::-1] for t in combos]
combos.extend(combos_backward)

In [9]:
warnings.simplefilter("ignore", RuntimeWarning)

matchups = pd.DataFrame()
for matchup in tqdm(combos):
    home_team = matchup[0]
    away_team = matchup[1]
    
    pred = pregame_stat_2(home_team, away_team, str(datetime.today()).split()[0], NBAgamesC, NBAgames, nba_team, 
                                  ELO_track, all_boxes, folded_out, folded_roster, 
                   num_games_for_avg = 10, current_season = 2021, upcoming = True, playoff = playoff)
    pred['Matchup'] = away_team + '@' + home_team
    
    matchups = pd.concat([matchups, pred])  

100%|██████████| 870/870 [04:42<00:00,  3.08it/s]


In [10]:
pd.DataFrame.to_csv(matchups, '/Users/nickbachelder/Buckets Model/matchups')

In [11]:
curr_inj = pd.DataFrame()
curr_inj['PLAYER'] = get_curr_injuries()
teams = []
for player in tqdm(curr_inj['PLAYER']):
    team_id_inj = unfolded_roster.loc[(unfolded_roster['SEASON'] == curr_season) & (unfolded_roster['PLAYER'] == player)]['TeamID'].reset_index(drop = 1)[0]
    team_abb_inj = nba_team.loc[nba_team['id'] == int(team_id_inj)]['abbreviation'].reset_index(drop = 1)[0]
    teams.append(team_abb_inj)
    

curr_inj['TEAM'] = teams
curr_inj

pd.DataFrame.to_csv(curr_inj, '/Users/nickbachelder/Buckets Model/curr_inj')

100%|██████████| 43/43 [00:00<00:00, 788.12it/s]


In [12]:
starters_predicted = pd.DataFrame()
for abb in tqdm(nba_team['abbreviation']):
    player_avgs = get_team_avgs_over_last_x_games(abb, date = str(datetime.today()).split()[0], num_games_for_avg = 10, 
                                folded_roster = folded_roster, NBAgamesC = NBAgamesC,
                                all_boxes = all_boxes, nba_team = nba_team, current_season = curr_season)
    player_avgs_top = player_avgs.loc[~player_avgs['PLAYER_NAME'].isin(curr_inj['PLAYER'])].nlargest(5, 'PTS')
    player_avgs_top['TEAM_ABB'] = abb
    starters_predicted = pd.concat([starters_predicted, player_avgs_top])
    
starters_predicted

pd.DataFrame.to_csv(starters_predicted, '/Users/nickbachelder/Buckets Model/starters_predicted')

100%|██████████| 30/30 [00:03<00:00,  8.47it/s]


In [13]:
# Pull new NBAgames C entries

def update_NBAgamesC():
    games = pd.DataFrame()
    gameappend_regular = endpoints.leaguegamefinder.LeagueGameFinder(season_type_nullable = SeasonType.regular, 
                                              season_nullable = '2021-22').get_data_frames()[0]
    gameappend_playoff = endpoints.leaguegamefinder.LeagueGameFinder(season_type_nullable = SeasonTypePlayoffs.playoffs, 
                                              season_nullable = '2021-22').get_data_frames()[0]
    gameappend_regular['PLAYOFF'] = 0 
    gameappend_playoff['PLAYOFF'] = 1
    games = games.append(pd.DataFrame(gameappend_regular))
    games = games.append(pd.DataFrame(gameappend_playoff))
    if games.empty:
        return('No games to update')
    games['WL'] = games['WL'].eq('W').mul(1)
    NBA_codes = nba_team['id']
    NBAgames22 = games[games['TEAM_ID'].isin(np.array(NBA_codes))]
    ## ALSO: Order by time, format times correctly
    NBAgames22 = NBAgames22.sort_values(by=['GAME_DATE'], ascending = 0).reset_index(drop = 1)
    NBAgames22['GAME_DATE'] = pd.to_datetime(NBAgames22['GAME_DATE'])
    NBAgames22['TEAM_ID'] = NBAgames22['TEAM_ID'].astype(str)
    NBAgames22 = NBAgames22[NBAgames22['GAME_DATE'] != datetime.today().strftime('%Y-%m-%d')]
    ### Now need to combine the same games listed twice for the home and away team
    ## Must filter to unique games by combinding home and away games (vs or @ in mathchup)
    NBAgamesC22 = pd.DataFrame()
    for game_id in tqdm(np.unique(NBAgames22['GAME_ID'])):
        game_match = pd.DataFrame(NBAgames22[NBAgames22['GAME_ID'] == game_id])
        home_game = pd.DataFrame(game_match[game_match['MATCHUP'].str.contains('vs')])
        away_game = pd.DataFrame(game_match[game_match['MATCHUP'].str.contains('@')]).drop(['SEASON_ID',
                                                                                        'GAME_DATE',
                                                                                        'GAME_ID'
                                                                                       ], axis =1)
        home_game.columns = [str(col) + '_H' for col in home_game.columns]
        away_game.columns = [str(col) + '_A' for col in away_game.columns]
        combined = pd.DataFrame(pd.concat([home_game.reset_index(drop=True), away_game.reset_index(drop=True)], axis = 1))
        NBAgamesC22 = pd.concat([NBAgamesC22, combined])
        
    # Find Team swithes and change to new team name
    NBAgamesC22 = NBAgamesC22.drop(['PLUS_MINUS_A','PLUS_MINUS_H'], axis = 1).replace({'NOH': 'NOP',
                              'NJN': 'BKN',
                              'SEA': 'OKC',
                              'VAN': 'MEM',
                              'KCK': 'SAC',
                              'CHH': 'CHA',
                              'SDC': 'LAC',
                              'UTH': 'UTA',
                              'SAN': 'SAS',
                              'GOS': 'GSW',
                              'PHL': 'PHI',
                              'NOH': 'NOP',
                              'New Orleans Hornets': 'New Orleans Pelicans',
                              'NOK': 'NOP',
                             'New Orleans/Oklahoma City Hornets': 'New Orleans Pelicans',
                              'New Orleans Hornets': 'New Orleans Pelicans',
                              'New Jersey Nets': 'Brooklyn Nets',
                              'Seattle SuperSonics': 'Oklahoma City Thunder',
                              'Vancouver Grizzlies': 'Memphis Grizzlies',
                              'Kansas City Kings': 'Sacramento Kings',
                              'San Diego Clippers': 'Los Angeles Clippers'}, regex=True).sort_values(by=['GAME_DATE_H'], 
                                                                                                     ascending = 0).reset_index(drop = 1)
    NBAgamesC22['SEASON_ID_H'] = NBAgamesC22['SEASON_ID_H'].str[1:]
    NBAgamesC22
    
new_games = update_NBAgamesC()

if type(new_games) == pd.core.frame.DataFrame:
    NBAgamesC = pd.concat([NBAgamesC, new_games])

new_games

'No games to update'

In [14]:
# Pull new all_boxes entries

def update_all_boxes():
    regular_boxes = endpoints.PlayerGameLogs(season_type_nullable = SeasonType.regular, 
                                              season_nullable = '2021-22').get_data_frames()[0]
        
    post_boxes = endpoints.PlayerGameLogs(season_type_nullable = SeasonTypePlayoffs.playoffs, 
                                              season_nullable = '2021-22').get_data_frames()[0]
    
    post_boxes['post'] = 1
    regular_boxes['post'] = 0
    boxes_year = pd.concat([regular_boxes, post_boxes])
    all_boxes22 = pd.concat([all_boxes, boxes_year])
    
    if boxes_year.empty:
        return('No boxes to update')
    
    
    all_boxes22['GAME_DATE'] = pd.to_datetime(all_boxes22['GAME_DATE'].str[:10])
    all_boxes22['GAME_ID'] = all_boxes22['GAME_ID'].astype(int)
    all_boxes22 = all_boxes22.loc[all_boxes22['MIN'] != 0] 
    
    all_boxes22 = all_boxes22[['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
                           'GAME_ID', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 
                           'FTM', 'FTA', 'FT_PCT', 'REB', 'AST', 'STL', 'BLK', 'PTS']]
    
new_boxes = update_all_boxes()

if type(new_boxes) == pd.core.frame.DataFrame:
    all_boxes = pd.concat([all_boxes, new_boxes])

new_boxes

'No boxes to update'

In [15]:
## update elo

def update_elo(new_games):
    if type(new_games) != pd.core.frame.DataFrame:
        return('No ELO updates yet this season')
    AllSeasonGames = pd.DataFrame(NBAgamesC22).iloc[::-1].reset_index(drop = 1) ## In correct order
    CurrSeason = AllSeasonGames.iloc[0,]['SEASON_ID_H']
    
    ELO_track = pd.DataFrame() ## Creating starting ELOs from the fivethirtyeight 1982 data
    for team in nba_team['abbreviation']:
        ELO_start = {'Team': team,
                     'ELO': get_pre_elo(team, str(datetime.today()).split()[0], NBAgamesC, ELO_track),
                     'Date': AllSeasonGames.iloc[0,]['GAME_DATE_H'],
                     'Season': CurrSeason}
        ELO_track = ELO_track.append(pd.DataFrame(ELO_start, index=[0]))
    date = AllSeasonGames.iloc[0,]['GAME_DATE_H']
    end_date = AllSeasonGames.iloc[-1,]['GAME_DATE_H']
    delta = dt.timedelta(days=1)
    seasonindex = 1
    ELO_track = ELO_track.reset_index(drop = 1)
    
    while date <= end_date:
        ThisDayGames = pd.DataFrame(AllSeasonGames[AllSeasonGames['GAME_DATE_H'] == date]).reset_index(drop = 1)
        if ThisDayGames.empty:
            date += delta
            continue
        if ThisDayGames[['SEASON_ID_H']].iloc[0,][0] != CurrSeason: 
            for team in np.array(nba_team['abbreviation']):
                old_ELO = ELO_track[ELO_track['Team'] == team][['ELO']].iloc[-1,:]
                new_ELO = (.75 * old_ELO) + (.25 * 1505)
                newELOadd = {'Team': team,
                             'ELO': new_ELO,
                             'Date': date - delta,
                             'Season': ThisDayGames[['SEASON_ID_H']].iloc[0,][0]
                             }
            CurrSeason = ThisDayGames[['SEASON_ID_H']].iloc[0,][0]
            print('New Season', 'Season', seasonindex, 'Complete', '. Start', CurrSeason)
            seasonindex+=1
        for index in range(len(ThisDayGames)):
            game = ThisDayGames.iloc[index,:]
            gamedate = game[['GAME_DATE_H']][0]
            gamescore_diff = abs(int(game['PTS_H'])-int(game['PTS_A']))
            
            team1_abb = game[['MATCHUP_H']][0][0:3]
            team2_abb = game[['MATCHUP_H']][0][-3:]
            team1 = nba_team.loc[nba_team['abbreviation'] == team1_abb]['full_name'].values[0]
            team2 = nba_team.loc[nba_team['abbreviation'] == team2_abb]['full_name'].values[0]
            team1id = nba_team.loc[nba_team['abbreviation'] == team1_abb]['id'].values[0]
            team2id = nba_team.loc[nba_team['abbreviation'] == team2_abb]['id'].values[0]
            
            
            t1W = game['WL_H'] 
            
            t1_ELO = ELO_track[ELO_track['Team'] == team1_abb].iloc[-1,:][['ELO']]
            t2_ELO = ELO_track[ELO_track['Team'] == team2_abb].iloc[-1,:][['ELO']]
    
            
            ## ELO FORMULA
            if game['WL_H'] == 1:
                S_teamT1 = 1
                S_teamT2 = 0
            else:
                S_teamT1 = 0
                S_teamT2 = 1
            
            if S_teamT1 == 1:
                ELO_diff = (t1_ELO+100) - t2_ELO ## 100 extra points for home team
            else:
                ELO_diff = t2_ELO - (t1_ELO+100)
                
            E_teamT1 = 1/(1 + 10**((t2_ELO-(t1_ELO+100))/400)) ## 100 extra points for home team
            E_teamT2 = 1/(1 + 10**(((t1_ELO+100)-t2_ELO)/400))
    
            kt1 = 20*( (gamescore_diff+3)**0.8 / (7.5 + 0.006*ELO_diff))
            kt2 = 20*( (gamescore_diff+3)**0.8 / (7.5 + 0.006*ELO_diff))
            # Compute new ELOs
            t1_ELO_new = kt1*(S_teamT1 - E_teamT1) + t1_ELO
            t2_ELO_new = kt2*(S_teamT2 - E_teamT2) + t2_ELO
            
            ELOt1_append = {'Team': team1_abb,
             'ELO': t1_ELO_new,
             'Date': gamedate,
                           'Season': CurrSeason}
            ELOt2_append = {'Team': team2_abb,
             'ELO': t2_ELO_new,
             'Date': gamedate,
                           'Season': CurrSeason}
            addtoELO = pd.concat([pd.DataFrame(ELOt1_append), pd.DataFrame(ELOt2_append)], axis = 0)
            ELO_track = ELO_track.append(addtoELO).reset_index(drop = 1)
        date += delta
        
new_elo = update_elo(new_games = new_games)

if type(new_elo) == pd.core.frame.DataFrame:
    ELO_track = pd.concat([ELO_track, new_elo])
    
new_elo

'No ELO updates yet this season'

In [16]:
pd.DataFrame.to_csv(all_boxes, '/Users/nickbachelder/Buckets Model/all_boxes')
pd.DataFrame.to_csv(NBAgamesC, '/Users/nickbachelder/Buckets Model/NBAgamesC')

In [17]:
## add betting lines

### pd.read_html()

In [18]:
### fix inflated stats at beginning of season since all players are fresh out 
### of the playoffs and have inflated stats. Also penalize stars at the beginning of season if they are
### playing with a bunch of new stars

### make sure elos updated between season

## add a strength of schedule for last 10 games variable?

#### new players from draft are not being counted at the beginning of the season!! Need to fix this somehow